# Stochastic Programs

>*stochastic,* adj.  
Etymology: < Greek στοχαστικός, < στοχάζεσθαι to aim at a mark, guess, < στόχος aim, guess....
Randomly determined; that follows some random probability distribution or pattern, so that its behaviour may be analysed statistically but not predicted precisely; *stochastic process = random process* n. (Oxford English Dictionary (Accessed November 7, 2018)

## What is a stochastic program?

>stochastic programs, i.e., programs that exploit randomness. (Guttag, John V. *Introduction to Computation and Programming Using Python: With Application to Understanding Data*) 

In [ ]:
import seaborn as sns
import random
import numpy as np
import numpy.random as ra
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from collections import defaultdict
import copy
import collections.abc as abc
warnings.filterwarnings('ignore')

## Flipping a Coin

Let's write some functions to simulate flipping a coin. Let's start off with the coin being either 0 or 1.

* Heads=1, tails=0

In [ ]:
def coin1(seed=None):
    random.seed(seed)
    return random.choice([0,1])
def coin2(seed=None):
    random.seed(seed)
    if random.random() <= 0.5:
        return 1
    else:
        return 0

####  Write new versions `coin3` and `coin4` that return "heads" or "tails"

In [ ]:
    
def coin3(seed=None):
    random.seed(seed)
    return random.choice(["heads","tails"])    

def coin4(seed=None):
    random.seed(seed)
    if random.random() <= 0.5:
        return 1
    else:
        return 0

#### Look at the results of lots of coin flips

In [ ]:
ax = sns.distplot([coin2() for i in range(1000)], axlabel="results", color=(0.8, 0.0, 0.2, 0.5), kde=False, norm_hist=False)
ax.set_xticks([0,1])
ax.set_xticklabels(["tails", "heads"])
ax.set_ylabel("Counts")

#### If we run the above cell multiple times, we'll see the counts jumping around 500

#### How can we quanity this?

In [ ]:
def get_probabilities(values):
    """
    Computes observed probabilities for distinct values in values
    
    Input:
        values--a collection of observed values
        
    Output:
        a dictionary with keys the observed values and values the observed probability for each
        value
    """
    obs = set(values)
    n = len(values)
    vs = np.array(values)
    return {o:np.sum(np.where(vs==o,1,0))/n for o in obs}

#### Quantify the Behavior

In [ ]:
rslts = [get_probabilities([coin3() for i in range(50)]) for i in range(10)]

#### Now we need to collate our heads and tails results

In [ ]:
rslts2 = defaultdict(list)
for r in rslts:
    for k,v in r.items():
        rslts2[k].append(v)

In [ ]:
rslts3 = pd.DataFrame.from_dict(rslts2)
rslts3.head()

#### Are our probabilities valid?

In [ ]:
rslts3["prob check"] = rslts3["heads"]+rslts3["tails"]
rslts3

In [ ]:
rslts3["heads"].mean(), rslts3["heads"].std()

#### Use the `describe` method to get summary statistics

In [ ]:
a = rslts3.describe()
a

## Face card class

### New concepts

* `class` vs `instance` attributes
* `@staticmethod`
* `__hash__`

In [ ]:
class card2(object):

    __values = {2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:10,"jack":11, "queen":12,"king":13, "ace":14}
    __valid_suits = frozenset(("clubs", "diamonds", "hearts", "spades"))
    __valid_values = frozenset(__values.keys()) 
    
    
    def __init__(self, value, suit, *args, **kwargs):
        
        if (value not in card2.__valid_values) or (suit not in card2.__valid_suits):
            raise ValueError("invalid value or suit provided")
        self.__value = value
        self.__suit = suit
        
    @staticmethod
    def class_suits():
        return card2.__valid_suits
    @staticmethod
    def class_values():
        return card2.__valid_values
    
    @property
    def value(self):
        return self.__value
    @property
    def suit(self):
        return self.__suit
    def __str__(self):
        return "<%s, %s>"%(self.value, self.suit)
    def __repr__(self):
        return self.__str__()
    def __lt__(self, other):
        return card2.__values[self.value] < card2.__values[other.value]
    def __gt__(self, other):
        return card2.__values[self.value] > card2.__values[other.value]
    def __ge__(self, other):
        return card2.__values[self.value] >= card2.__values[other.value] 
    def __le__(self, other):
        return card2.__values[self.value] <= card2.__values[other.value]
    def __eq__(self, other):
        return card2.__values[self.value] == card2.__values[other.value]
    def __add__(self, other):
        return card2.__values[self.value] + card2.__values[other.value]
    
    def __hash__(self):
        return hash(self.__repr__())

In [ ]:
card2.class_suits()

In [ ]:
deck = [card2(v,s) for v in card2.class_values() for s in card2.class_suits()]

In [ ]:
print(deck)

## Make a `CardDeck` class by subtyping a `list`

#### See [this](https://stackoverflow.com/questions/9432719/python-how-can-i-inherit-from-the-built-in-list-type) discussion

#### Concepts

* `__new__`


In [ ]:
class CardDeck(list):
    def __new__(cls, *args, **kwargs):
        if not all([isinstance(a,card2) for a in args[0]]):
            raise TypeError("CardDeck can only contain cards")
        obj = super().__new__(cls, *args, **kwargs)
        return obj
    def __str__(self):
        tmp = [str(v) for v in self]
        tmp.sort()
        width = max([len(t) for t in tmp]) + 5
        num_per_row = int(85/width)
        rslt = ""
        for i in range(len(tmp)):
            
            rslt = rslt + tmp[i].ljust(width)
            if (i+1) % num_per_row == 0:
                rslt = rslt + "\n"
        return rslt
    def __repr__(self):
        tmp = [str(v) for v in self]
        return ",".join(tmp)

        
        

In [ ]:
deck2 = CardDeck([card2(v,s) for v in card2.class_values() for s in card2.class_suits()])
deck2.sort()

In [ ]:
a = deck2[0]
b = deck2[-1]

In [ ]:
a < b


In [ ]:
print(deck2)

In [ ]:
random.shuffle(deck2)
deck2

In [ ]:
print(deck2)

## `RandomEvent` Class

#### New concepts

* [Abstract Collections](https://docs.python.org/3/library/collections.abc.html)

In [ ]:
class RandomEvent(object):
    
    @staticmethod
    def __cum_prob(vals):
        assert isinstance(vals, abc.Mapping)

        items = list(vals.items())
        cum_prob = [items.pop(0)]
        for item in items:
            cum_prob.append((item[0], item[1]+cum_prob[-1][1] ))
            print(cum_prob)
        return cum_prob
    
    def __init__(self, vals, name="", seed=None):
        
        random.seed(seed)
        
        self.__name = name
        if isinstance(vals, abc.Mapping):
            
            self.__select = self.__select_cum_prob
            self.__cum_prob = RandomEvent.__cum_prob(vals)
            self.__vals = list(vals.values())
        elif isinstance(vals, abc.Container):
            self.__select = self.__select_choice
            self.__cum_prob = None
            self.__vals = copy.copy(vals)
        else:
            raise TypeError("non container type provided")
        
            
    @property
    def name(self):
        return self.__name
    
    def event(self):
        return self.__select()
    def __select_choice(self):
        return random.choice(self.__vals)
    def __select_cum_prob(self):
        
        val = random.random()
        for event, cp in self.__cum_prob:
            if val < cp:
                return event
        raise Exception("Something went wrong")
        
    def __str__(self):
        rslt = ""
        tmp = list(set(self.__vals))
        tmp.sort()
        tmp = [str(t) for t in tmp]
        
        width = max([len(t) for t in tmp])+5
        num_per_row = int(85/width)
        for i in range(len(tmp)):
            rslt = rslt + tmp[i].ljust(width)
            if (i+1)%num_per_row==0:
                rslt = rslt + "\n"
            
        return "%s:\n%s"%(self.__name, rslt) 

In [ ]:
process = RandomEvent(vals=deck, name="playing cards")

In [ ]:
str(process.event())

In [ ]:
print(process)

#### Modify `__cum_prob` to raise a `ValueError` if the probabilities are in valid.

#### Add a `get_probabilities` method to the `RandomEvent` Class

```Python
def get_probabilities(values):
    """
    Computes observed probabilities for distinct values in values
    
    Input:
        values--a collection of observed values
        
    Output:
        a dictionary with keys the observed values and values the observed probability for each
        value
    """
    obs = set(values)
    n = len(values)
    vs = np.array(values)
    return {o:np.sum(np.where(vs==o,1,0))/n for o in obs}
```

#### Based on our earlier code write a method `characterize` 

* returns a Pandas DataFrame summarizing probabilities based on two arguments:
    * `m` number of events per experiment
    * `n` number of experiments
    
```Python
rslts = [get_probabilities([coin3() for i in range(50)]) for i in range(10)]

rslts2 = defaultdict(list)
for r in rslts:
    for k,v in r.items():
        rslts2[k].append(v)
        
rslts3 = pd.DataFrame.from_dict(rslts2)

a = rslts3.describe()
```

## Make a six-sided fair `die` RandomEvent

In [ ]:
die = RandomEvent(name="die", vals=[1,2,3,4,5,6])
print(die)

In [ ]:
sns.distplot([die.event() for i in range(1000)], kde=False)

## Make a random die and estimate it's characteristics

In [ ]:
die2 = RandomEvent(name="unknown die", vals=ra.randint(1,20, size=10))
sns.distplot([die2.event() for i in range(1000)], kde=False)

## Modeling Population Age Distribution

Below is a [plot](https://en.wikipedia.org/wiki/Demography_of_the_United_States) of the ages of people in the United States.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/03/USA_by_Sex_and_Age_2015-07-01.svg/700px-USA_by_Sex_and_Age_2015-07-01.svg.png" alt="USA age distribution"  width="300">

Last year a homework assignment required students to model a population of students. As a first approximation, we used the [geometric distribution](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.random.geometric.html#numpy.random.geometric) to estimate the starting age distribution for a population.

In [ ]:
def get_age(minage=17, maxage=100, p=0.06):
    age = minage+ra.geometric(p,1)[0]
    if age < maxage:
        return age
    else:
        return get_age()
ages = pd.Series([get_age() for i in range(100000)])
print(ages.min(), ages.max())
ages.hist(bins=100)

### Play Around!

Explore some of the other probability distributions defined in [numpy.random](https://docs.scipy.org/doc/numpy-1.15.1/reference/routines.random.html) to see if you can come up with a distribution you feel would be a better fit. Assume population ranges from 0 to 100.

In [ ]:
mu, sigma = -15, 0.6
tmp = ra.lognormal(mu, sigma, 1000)
tmp = tmp - tmp.min()
sns.distplot(tmp*(100.0/np.max(tmp)))

In [ ]:
alpha, beta = 1.5, 6
tmp = ra.beta(alpha, beta, 1000)
tmp = tmp - tmp.min()
sns.distplot(tmp*(100.0/np.max(tmp)))

In [ ]:
a=1.5
tmp = ra.weibull(a, 1000)
tmp = tmp - tmp.min()
sns.distplot(tmp*(100.0/np.max(tmp)))